Instruções
O desafio será construir um video a partir de imagens geradas pelo Stable Diffusion e que deverão ser retroalimentadas no modelo para gerar o próximo frame.

In [ ]:
# Instalação
%pip -q install diffusers==0.30.0 transformers accelerate safetensors "imageio[ffmpeg]" pillow


In [ ]:
# 🎬 MOVIMENTO: Strength Progressivo
import os, numpy as np
from PIL import Image
import torch, imageio
from diffusers import StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler
import time

print("🎬 Iniciando CONTINUIDADE + MOVIMENTO...")
start_time = time.time()

# Configuração para continuidade + movimento
model_id = "runwayml/stable-diffusion-v1-5"
width, height = 512, 512
width  = (width  // 8) * 8
height = (height // 8) * 8
steps, guidance, frames, fps = 20, 7.5, 8, 12
negative_prompt = "blurry, low quality, distorted, deformed, ugly, disfigured, mutated, extra limbs, missing limbs, bad anatomy, bad proportions, malformed, different character, not Homer Simpson"

print(f"📊 Config: {width}x{height}, {steps} steps, {frames} frames")

# Carregar imagem
img_path = "/Users/alansms/stable_diffusion_video_generator/homer.png"
if not os.path.exists(img_path):
    raise FileNotFoundError(f"Imagem não encontrada: {img_path}")
init_img = Image.open(img_path).convert("RGB").resize((width, height), Image.LANCZOS)
print(f"✅ Homer carregado: {init_img.size}")

# Pipeline APENAS Img2Img para continuidade
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)

if device == "cuda":
    pipe.enable_attention_slicing()

print("✅ Pipeline de continuidade pronto")

# SEQUÊNCIA PROGRESSIVA: Movimento crescente com continuidade
progressive_sequence = [
    ("Homer Simpson, yellow skin, cartoon character, arms slightly lowered, same expression", 0.6),
    ("Homer Simpson, yellow skin, cartoon character, arms down, relaxed pose, gentle movement", 0.65),
    ("Homer Simpson, yellow skin, cartoon character, right arm pointing forward, left arm down", 0.7),
    ("Homer Simpson, yellow skin, cartoon character, both arms raised to shoulder height, excited", 0.75),
    ("Homer Simpson, yellow skin, cartoon character, both arms raised high, very excited, jumping", 0.8),
    ("Homer Simpson, yellow skin, cartoon character, hands on hips, confident pose", 0.75),
    ("Homer Simpson, yellow skin, cartoon character, arms crossed, thinking pose", 0.7),
    ("Homer Simpson, yellow skin, cartoon character, original pose, arms raised, returning", 0.65)
]

frames_list = [init_img]
current = init_img

print(f"🎞️ Gerando {frames} frames com CONTINUIDADE + MOVIMENTO...")
for i in range(1, frames):
    frame_start = time.time()
    prompt, strength = progressive_sequence[i-1]
    print(f"  Frame {i+1}/{frames} (strength={strength}): {prompt[:40]}...", end=" ")
    
    out = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=current,
        strength=strength,  # ← Strength progressivo
        num_inference_steps=steps,
        guidance_scale=guidance,
    )
    
    img = out.images[0]
    if not isinstance(img, Image.Image):
        img = Image.fromarray(np.array(img)).convert("RGB")
    
    frames_list.append(img)
    current = img
    
    frame_time = time.time() - frame_start
    print(f"✅ ({frame_time:.1f}s)")

# Exportar
os.makedirs("output", exist_ok=True)
out_path = "output/homer_continuidade_movimento.mp4"
with imageio.get_writer(out_path, fps=fps, quality=8) as w:
    for im in frames_list:
        w.append_data(np.array(im.convert("RGB")))

total_time = time.time() - start_time
print(f"🎉 CONCLUÍDO em {total_time:.1f} segundos!")
print(f"✅ Vídeo salvo: {out_path}")
print(f"📊 {len(frames_list)} frames, {fps} FPS, {width}x{height}px")
print(f"⚡ Tempo médio por frame: {total_time/frames:.1f}s")
print("🎬 Resultado: Homer com CONTINUIDADE + MOVIMENTO REAL!")


🎬 Iniciando CONTINUIDADE + MOVIMENTO...
📊 Config: 512x512, 20 steps, 8 frames
✅ Homer carregado: (512, 512)


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 30.12it/s]


✅ Pipeline de continuidade pronto
🎞️ Gerando 8 frames com CONTINUIDADE + MOVIMENTO...
  Frame 2/8 (strength=0.6): Homer Simpson, yellow skin, cartoon char... 

100%|██████████| 12/12 [00:37<00:00,  3.13s/it]


✅ (44.8s)
  Frame 3/8 (strength=0.65): Homer Simpson, yellow skin, cartoon char... 

...

✅ (49.4s)
  Frame 7/8 (strength=0.75): Homer Simpson, yellow skin, cartoon char... 

100%|██████████| 15/15 [00:41<00:00,  2.80s/it]


✅ (49.4s)
  Frame 8/8 (strength=0.7): Homer Simpson, yellow skin, cartoon char... 

100%|██████████| 14/14 [00:39<00:00,  2.82s/it]


✅ (46.5s)
🎉 CONCLUÍDO em 329.2 segundos!
✅ Vídeo salvo: output/homer_continuidade_movimento.mp4
📊 8 frames, 12 FPS, 512x512px
⚡ Tempo médio por frame: 41.1s
🎬 Resultado: Homer com CONTINUIDADE + MOVIMENTO REAL!


In [1]:
# 🎭 MÉTODO AVANÇADO: Interpolação entre poses-chave
import os, numpy as np
from PIL import Image
import torch, imageio
from diffusers import StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler
import time

print("🎭 Iniciando MÉTODO AVANÇADO com interpolação...")
start_time = time.time()

# Configuração avançada
model_id = "runwayml/stable-diffusion-v1-5"
width, height = 512, 512
width  = (width  // 8) * 8
height = (height // 8) * 8
steps, guidance, frames, fps = 25, 8.0, 10, 12
negative_prompt = "blurry, low quality, distorted, deformed, ugly, disfigured, mutated, extra limbs, missing limbs, bad anatomy, bad proportions, malformed, different character, not Homer Simpson"

print(f"📊 Config: {width}x{height}, {steps} steps, {frames} frames")

# Carregar Homer
img_path = "/Users/alansms/stable_diffusion_video_generator/homer.png"
if not os.path.exists(img_path):
    raise FileNotFoundError(f"Imagem não encontrada: {img_path}")
init_img = Image.open(img_path).convert("RGB").resize((width, height), Image.LANCZOS)
print(f"✅ Homer carregado: {init_img.size}")

# Pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)

if device == "cuda":
    pipe.enable_attention_slicing()

print("✅ Pipeline avançado pronto")

# INTERPOLAÇÃO ENTRE POSES-CHAVE
key_poses = [
    ("Homer Simpson, yellow skin, cartoon character, original pose, arms raised", 0.5),
    ("Homer Simpson, yellow skin, cartoon character, arms slightly lowered, transition", 0.6),
    ("Homer Simpson, yellow skin, cartoon character, arms down, relaxed, gentle movement", 0.7),
    ("Homer Simpson, yellow skin, cartoon character, right arm pointing, left arm down", 0.75),
    ("Homer Simpson, yellow skin, cartoon character, both arms to shoulders, excited", 0.8),
    ("Homer Simpson, yellow skin, cartoon character, both arms raised high, very excited", 0.8),
    ("Homer Simpson, yellow skin, cartoon character, hands on hips, confident", 0.75),
    ("Homer Simpson, yellow skin, cartoon character, arms crossed, thinking", 0.7),
    ("Homer Simpson, yellow skin, cartoon character, returning to original, arms up", 0.6),
    ("Homer Simpson, yellow skin, cartoon character, original pose, arms raised", 0.5)
]

frames_list = [init_img]
current = init_img

print(f"🎞️ Gerando {frames} frames com INTERPOLAÇÃO AVANÇADA...")
for i in range(1, frames):
    frame_start = time.time()
    prompt, strength = key_poses[i-1]
    print(f"  Frame {i+1}/{frames} (strength={strength}): {prompt[:50]}...", end=" ")
    
    out = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=current,
        strength=strength,
        num_inference_steps=steps,
        guidance_scale=guidance,
    )
    
    img = out.images[0]
    if not isinstance(img, Image.Image):
        img = Image.fromarray(np.array(img)).convert("RGB")
    
    frames_list.append(img)
    current = img
    
    frame_time = time.time() - frame_start
    print(f"✅ ({frame_time:.1f}s)")

# Exportar
os.makedirs("output", exist_ok=True)
out_path = "output/homer_interpolacao_avancada.mp4"
with imageio.get_writer(out_path, fps=fps, quality=8) as w:
    for im in frames_list:
        w.append_data(np.array(im.convert("RGB")))

total_time = time.time() - start_time
print(f"🎉 CONCLUÍDO em {total_time:.1f} segundos!")
print(f"✅ Vídeo salvo: {out_path}")
print(f"📊 {len(frames_list)} frames, {fps} FPS, {width}x{height}px")
print(f"⚡ Tempo médio por frame: {total_time/frames:.1f}s")
print("🎭 Resultado: Homer com INTERPOLAÇÃO AVANÇADA e continuidade!")


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🎭 Iniciando MÉTODO AVANÇADO com interpolação...
📊 Config: 512x512, 25 steps, 10 frames
✅ Homer carregado: (512, 512)


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 34.48it/s]


✅ Pipeline avançado pronto
🎞️ Gerando 10 frames com INTERPOLAÇÃO AVANÇADA...
  Frame 2/10 (strength=0.5): Homer Simpson, yellow skin, cartoon character, ori... 

100%|██████████| 12/12 [00:36<00:00,  3.07s/it]


...

✅ (56.8s)
  Frame 9/10 (strength=0.7): Homer Simpson, yellow skin, cartoon character, arm... 

100%|██████████| 17/17 [00:46<00:00,  2.74s/it]


✅ (53.7s)
  Frame 10/10 (strength=0.6): Homer Simpson, yellow skin, cartoon character, ret... 

100%|██████████| 15/15 [00:42<00:00,  2.81s/it]


✅ (48.9s)
🎉 CONCLUÍDO em 489.3 segundos!
✅ Vídeo salvo: output/homer_interpolacao_avancada.mp4
📊 10 frames, 12 FPS, 512x512px
⚡ Tempo médio por frame: 48.9s
🎭 Resultado: Homer com INTERPOLAÇÃO AVANÇADA e continuidade!
